# Reasoning

## Why was this chosen?

Unfortunately, with the AI *.ipynb* files losing working code too fast at the moment, it means copilot has a lot of old flawed examples to help with. It will take more than the hackathon time to get the right solution together using the model I wanted to use.

That being said, there are at least models that can fit [meta-llama/Llama-3.2-1B](https://huggingface.co/meta-llama/Llama-3.2-1B) should be a good model to train as the assistant (training in the future should be fine from copliot generated code in *01*, just needs work, mostly on prompts my side). We shall see in the future.

In [ ]:
%%time
!pip install -q --upgrade trl --break-system-packages

In [ ]:
%%time
!pip install -q accelerate --break-system-packages

In [1]:
%%time
# Hide Warnings
import warnings
warnings.filterwarnings('ignore')

CPU times: user 26 µs, sys: 6 µs, total: 32 µs
Wall time: 35.5 µs


In [2]:
%%time
MODEL_NAME = 'NousResearch/Llama-3.2-1B'

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.44 µs


In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

In [4]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
%%time
# Load the Model
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

2024-11-13 07:44:52.412385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 07:44:52.425447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 07:44:52.429425: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 07:44:52.440496: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 07:44:59.275499: W tensorflow/compiler/tf2

CPU times: user 11.2 s, sys: 6.48 s, total: 17.7 s
Wall time: 1min 50s


In [6]:
%%time
for i in model.named_parameters():
    print(f"{i[0]} -> {i[1].device}")

model.embed_tokens.weight -> cuda:0
model.layers.0.self_attn.q_proj.weight -> cuda:0
model.layers.0.self_attn.k_proj.weight -> cuda:0
model.layers.0.self_attn.v_proj.weight -> cuda:0
model.layers.0.self_attn.o_proj.weight -> cuda:0
model.layers.0.mlp.gate_proj.weight -> cuda:0
model.layers.0.mlp.up_proj.weight -> cuda:0
model.layers.0.mlp.down_proj.weight -> cuda:0
model.layers.0.input_layernorm.weight -> cuda:0
model.layers.0.post_attention_layernorm.weight -> cuda:0
model.layers.1.self_attn.q_proj.weight -> cuda:0
model.layers.1.self_attn.k_proj.weight -> cuda:0
model.layers.1.self_attn.v_proj.weight -> cuda:0
model.layers.1.self_attn.o_proj.weight -> cuda:0
model.layers.1.mlp.gate_proj.weight -> cuda:0
model.layers.1.mlp.up_proj.weight -> cuda:0
model.layers.1.mlp.down_proj.weight -> cuda:0
model.layers.1.input_layernorm.weight -> cuda:0
model.layers.1.post_attention_layernorm.weight -> cuda:0
model.layers.2.self_attn.q_proj.weight -> cuda:0
model.layers.2.self_attn.k_proj.weight ->

In [7]:
%%time
input_text = 'Im not having a great day, I did my exercises, and suggested theropy, but now Im bored. What can I try out that can engage my mind, and keep me entertained?'
input_ids = tokenizer(input_text, return_tensors='pt').input_ids

CPU times: user 2.16 ms, sys: 37.4 ms, total: 39.6 ms
Wall time: 2.27 s


In [8]:
%%time
input_ids = input_ids.to(device)
generated_ids = model.generate(input_ids, max_length=1024)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CPU times: user 35.2 s, sys: 1.13 s, total: 36.4 s
Wall time: 1min 18s


In [9]:
%%time
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(generated_text)

Im not having a great day, I did my exercises, and suggested theropy, but now Im bored. What can I try out that can engage my mind, and keep me entertained? I have a few ideas, but I want to know what you guys think.
I have a few ideas, but I want to know what you guys think.
I have a few ideas, but I want to know what you guys think. I have a few ideas, but I want to know what you guys think.
I have a few ideas, but I want to know what you guys think. I have a few ideas, but I want to know what you guys think.
I have a few ideas, but I want to know what you guys think. I have a few ideas, but I want to know what you guys think.
I have a few ideas, but I want to know what you guys think. I have a few ideas, but I want to know what you guys think.
I have a few ideas, but I want to know what you guys think. I have a few ideas, but I want to know what you guys think.
I have a few ideas, but I want to know what you guys think. I have a few ideas, but I want to know what you guys think.
I h